# IVVI_rack Signal Chain Example

This notebook demonstrates the modular QCoDeS architecture for the IVVI_rack signal chain with current-setpoint control.

## Topology

```
[MFLI AC Voltage Source] → [Manual V→I Transformer] → [Sample] → [Manual Voltage Preamp] → [MFLI Input & Demod]
```

The system provides:
- Single current setpoint parameter (I_set) at the top level
- Open-loop current control via transconductance (gm)
- Coupled frequency control for source and lock-in
- Guard functions for input overload protection
- Derived physics parameters for monitoring

In [ ]:
# Import the necessary modules
import warnings
import numpy as np

# Mock QCoDeS for demonstration (normally you would just import qcodes)
class MockParameter:
    def __init__(self, initial_value=0.0, name="param"):
        self._value = initial_value
        self.name = name
    
    def __call__(self):
        return self._value
    
    def set(self, value):
        print(f"Setting {self.name} = {value}")
        self._value = value
    
    def get(self):
        return self._value

class MockInstrument:
    def __init__(self, name):
        self.name = name
        print(f"Created mock instrument: {name}")
    
    def add_parameter(self, name, **kwargs):
        pass

In [ ]:
# Create mock MFLI driver (normally this would be your actual MFLI driver)
class MockMFLI(MockInstrument):
    def __init__(self, name="mfli"):
        super().__init__(name)
        # Source parameters
        self.sigout_amplitude0 = MockParameter(0.0, "amplitude")
        self.sigout_enable0 = MockParameter(False, "output_enable")
        self.frequency = MockParameter(1000.0, "frequency")
        
        # Lock-in parameters
        self.time_constant = MockParameter(0.1, "time_constant")
        self.sensitivity = MockParameter(1.0, "sensitivity")
        self.sigout_range = MockParameter(1.0, "input_range")
        
        # Readouts
        self.X = MockParameter(0.0, "X")
        self.Y = MockParameter(0.0, "Y")
        self.R = MockParameter(0.0, "R")
        self.Theta = MockParameter(0.0, "Theta")

# Create the MFLI driver instance
mfli = MockMFLI("mfli_device")

In [ ]:
# Import and create the signal chain nodes
# In a real setup, you would import from qcodes_contrib_drivers.nodes

# For this demo, we'll use simplified mock versions
class MockMFLISource(MockInstrument):
    def __init__(self, mfli_driver, name="mfli_source"):
        super().__init__(name)
        self._mfli = mfli_driver
        self.amplitude = mfli_driver.sigout_amplitude0
        self.level = self.amplitude  # Alias
        self.output_on = mfli_driver.sigout_enable0
        self.frequency = mfli_driver.frequency

class MockManualVITransformer(MockInstrument):
    def __init__(self, name="vi_transformer"):
        super().__init__(name)
        self.gm_a_per_v = MockParameter(1e-3, "transconductance")  # 1 mA/V
        self.invert = MockParameter(False, "vi_invert")

class MockManualVoltagePreamp(MockInstrument):
    def __init__(self, name="voltage_preamp"):
        super().__init__(name)
        self.gain_v_per_v = MockParameter(100.0, "preamp_gain")  # 100 V/V
        self.invert = MockParameter(False, "preamp_invert")

class MockMFLILockIn(MockInstrument):
    def __init__(self, mfli_driver, name="mfli_lockin"):
        super().__init__(name)
        self._mfli = mfli_driver
        self.frequency = mfli_driver.frequency
        self.time_constant = mfli_driver.time_constant
        self.sensitivity = mfli_driver.sensitivity
        self.input_range = mfli_driver.sigout_range
        self.X = mfli_driver.X
        self.Y = mfli_driver.Y
        self.R = mfli_driver.R
        self.Theta = mfli_driver.Theta

# Create the device nodes
src_v = MockMFLISource(mfli)
vi = MockManualVITransformer()
preamp = MockManualVoltagePreamp()
lockin = MockMFLILockIn(mfli)

In [ ]:
# Create the SignalChain virtual instrument
# Simplified version for demo

class DemoSignalChain(MockInstrument):
    def __init__(self, src_v, vi, preamp, lockin, name="signal_chain"):
        super().__init__(name)
        self.src_v = src_v
        self.vi = vi
        self.preamp = preamp
        self.lockin = lockin
        
        # Flattened parameters (normally these would be DelegateParameters)
        self.excitation_v_ac = src_v.level
        self.output_on = src_v.output_on
        self.gm_a_per_v = vi.gm_a_per_v
        self.vi_invert = vi.invert
        self.preamp_gain = preamp.gain_v_per_v
        self.preamp_invert = preamp.invert
        
        # Manual advisory parameters
        self.R_est = MockParameter(None, "R_est")  # Sample resistance estimate
        self.margin = MockParameter(3.0, "margin")  # Safety margin
        
        # Lock-in parameters
        self.time_constant = lockin.time_constant
        self.sensitivity = lockin.sensitivity
        self.input_range = lockin.input_range
        self.X = lockin.X
        self.Y = lockin.Y
        self.R = lockin.R
        self.Theta = lockin.Theta
    
    def set_reference_frequency(self, f):
        """Set both source and lock-in frequency."""
        print(f"Setting reference frequency to {f} Hz")
        if hasattr(self.src_v, 'frequency') and self.src_v.frequency is not None:
            self.src_v.frequency.set(f)
        self.lockin.frequency.set(f)
    
    def get_reference_frequency(self):
        """Get lock-in frequency as reference."""
        return float(self.lockin.frequency())
    
    def _gm_eff(self):
        """Effective transconductance including inversion."""
        gm = float(self.gm_a_per_v())
        invert = bool(self.vi_invert())
        return -gm if invert else gm
    
    def _gv_eff(self):
        """Effective preamp gain including inversion."""
        gv = float(self.preamp_gain())
        invert = bool(self.preamp_invert())
        return -gv if invert else gv
    
    def _guard_lockin_overload(self, I_target):
        """Check for potential lock-in input overload and warn."""
        R_est = self.R_est()
        if R_est in (None, 0):
            return  # Cannot predict without resistance estimate
        
        # Predict preamp output voltage
        V_preamp_out_pred = abs(I_target) * float(R_est) * abs(self._gv_eff())
        
        # Check against input range
        input_range = float(self.input_range())
        threshold = 0.8 * input_range
        
        if V_preamp_out_pred > threshold:
            warnings.warn(
                f"[GUARD] Predicted lock-in input {V_preamp_out_pred:.3g} V "
                f"exceeds 80% of input range {input_range:.3g} V. "
                f"Consider reducing current or increasing input range.",
                UserWarning
            )
    
    def set_I_target(self, I_target):
        """Set target current by computing required source voltage."""
        print(f"\nSetting target current: {I_target:.3e} A")
        
        gm_eff = self._gm_eff()
        if gm_eff == 0:
            raise ValueError("gm_a_per_v is zero; cannot compute source voltage.")
        
        V_needed = I_target / gm_eff
        print(f"Computed required source voltage: {V_needed:.3e} V")
        
        # Optional guard check
        self._guard_lockin_overload(I_target)
        
        # Set source voltage and ensure output is on
        self.excitation_v_ac.set(V_needed)
        self.output_on.set(True)
        print(f"Source output enabled")
    
    def get_I_cmd(self):
        """Get commanded current based on current source voltage."""
        return self._gm_eff() * float(self.excitation_v_ac())
    
    def get_V_sample_ac_meas(self):
        """Get measured AC voltage at sample (complex)."""
        X_val = float(self.X())
        Y_val = float(self.Y())
        gv_eff = self._gv_eff()
        if gv_eff == 0:
            return complex(0, 0)
        return complex(X_val, Y_val) / gv_eff
    
    def get_I_meas(self):
        """Get measured current based on sample voltage and resistance."""
        R_est = self.R_est()
        if R_est in (None, 0):
            return None
        V_sample = self.get_V_sample_ac_meas()
        return abs(V_sample) / float(R_est)
    
    def get_recommended_sensitivity(self):
        """Get recommended sensitivity based on current R measurement."""
        margin = float(self.margin())
        R_val = float(self.R())
        return margin * R_val
    
    def get_topology_summary(self):
        """Get a summary of the signal chain topology and current settings."""
        gm_eff = self._gm_eff()
        gv_eff = self._gv_eff()
        
        summary = f"""
Signal Chain Topology Summary:
==============================
[MFLI Source] → [V→I Transformer] → [Sample] → [Voltage Preamp] → [MFLI Lock-in]

Current Settings:
- Source voltage (RMS): {self.excitation_v_ac()} V
- Output enabled: {self.output_on()}
- Reference frequency: {self.get_reference_frequency()} Hz
- Effective transconductance: {gm_eff:.3e} A/V
- Effective preamp gain: {gv_eff:.1f} V/V
- Estimated sample resistance: {self.R_est()} Ω

Derived Values:
- Commanded current: {self.get_I_cmd():.3e} A
- Measured current: {self.get_I_meas()} A (if R_est set)
- Sample voltage (complex): {self.get_V_sample_ac_meas()} V
"""
        return summary

# Create the signal chain
signal_chain = DemoSignalChain(src_v, vi, preamp, lockin)

## Basic Configuration

Let's configure the signal chain with typical lab parameters:

In [ ]:
# Configure the signal chain parameters
print("=== Configuring Signal Chain ===")

# Set transconductance for V→I converter (e.g., 1 mA/V)
signal_chain.gm_a_per_v.set(1e-3)  # 1 mA/V

# Set preamp gain (e.g., 100 V/V)
signal_chain.preamp_gain.set(100.0)  # 100 V/V

# Set sample resistance estimate (for I_meas calculation and guards)
signal_chain.R_est.set(10e3)  # 10 kΩ

# Set lock-in parameters
signal_chain.time_constant.set(0.1)  # 100 ms
signal_chain.sensitivity.set(1.0)    # 1 V sensitivity
signal_chain.input_range.set(1.0)    # 1 V input range

# Set reference frequency (couples source and lock-in)
signal_chain.set_reference_frequency(1000.0)  # 1 kHz

print("\nConfiguration complete!")

## Current Setpoint Control

Now let's demonstrate the main feature: setting a target current and having the system automatically compute the required source voltage.

In [ ]:
# Demonstrate current setpoint control
print("=== Current Setpoint Control Demo ===")

# Set target current to 1 µA
I_target = 1e-6  # 1 µA
signal_chain.set_I_target(I_target)

print(f"\nVerification:")
print(f"- Target current: {I_target:.3e} A")
print(f"- Commanded current: {signal_chain.get_I_cmd():.3e} A")
print(f"- Source voltage set to: {signal_chain.excitation_v_ac():.3e} V")
print(f"- Output enabled: {signal_chain.output_on()}")

# The math: I_target / gm_eff = V_needed
# 1e-6 A / 1e-3 A/V = 1e-3 V
print(f"\nExpected calculation: {I_target:.3e} A / {signal_chain._gm_eff():.3e} A/V = {I_target/signal_chain._gm_eff():.3e} V")

## Frequency Coupling

Demonstrate that setting the reference frequency updates both source and lock-in:

In [ ]:
print("=== Frequency Coupling Demo ===")

# Show current frequencies
print(f"Current source frequency: {signal_chain.src_v.frequency()} Hz")
print(f"Current lock-in frequency: {signal_chain.lockin.frequency()} Hz")

# Set new reference frequency
new_freq = 2500.0  # 2.5 kHz
signal_chain.set_reference_frequency(new_freq)

print(f"\nAfter setting reference frequency to {new_freq} Hz:")
print(f"Source frequency: {signal_chain.src_v.frequency()} Hz")
print(f"Lock-in frequency: {signal_chain.lockin.frequency()} Hz")
print(f"Reference frequency: {signal_chain.get_reference_frequency()} Hz")

## Guard Functions

Demonstrate the guard function that warns when predicted input might overload the lock-in:

In [ ]:
print("=== Guard Function Demo ===")

# Set up a scenario that will trigger the guard warning
# Current config: R_est=10kΩ, preamp_gain=100 V/V, input_range=1V
# For I_target=1µA: predicted V_preamp = 1e-6 * 10e3 * 100 = 1.0 V
# This equals the input range, so it will warn (threshold is 80% = 0.8V)

print("Testing guard with current settings...")
print(f"R_est = {signal_chain.R_est()} Ω")
print(f"Preamp gain = {signal_chain.preamp_gain()} V/V")
print(f"Input range = {signal_chain.input_range()} V")

# This should trigger a warning
with warnings.catch_warnings(record=True) as w:
    warnings.simplefilter("always")
    signal_chain.set_I_target(1e-6)  # 1 µA
    
    if w:
        print(f"\nGuard warning triggered: {w[0].message}")
    else:
        print("\nNo guard warning (current might be within safe range)")

# Try with a smaller current that should not trigger warning
print("\nTesting with smaller current...")
with warnings.catch_warnings(record=True) as w:
    warnings.simplefilter("always")
    signal_chain.set_I_target(0.5e-6)  # 0.5 µA
    
    if w:
        print(f"Guard warning: {w[0].message}")
    else:
        print("No guard warning - current is within safe range")

## Derived Parameters and Measurements

Simulate some measurements and show derived parameters:

In [ ]:
print("=== Derived Parameters Demo ===")

# Simulate some lock-in readouts (in practice these would come from real measurements)
signal_chain.X.set(1.0)   # 1 V X component
signal_chain.Y.set(0.0)   # 0 V Y component  
signal_chain.R.set(1.0)   # 1 V magnitude
signal_chain.Theta.set(0.0)  # 0° phase

print("Simulated lock-in readouts:")
print(f"X = {signal_chain.X()} V")
print(f"Y = {signal_chain.Y()} V")
print(f"R = {signal_chain.R()} V")
print(f"Theta = {signal_chain.Theta()} deg")

# Calculate derived parameters
V_sample = signal_chain.get_V_sample_ac_meas()
I_meas = signal_chain.get_I_meas()
recommended_sens = signal_chain.get_recommended_sensitivity()

print(f"\nDerived parameters:")
print(f"Sample voltage (complex): {V_sample} V")
print(f"Sample voltage magnitude: {abs(V_sample):.3e} V")
print(f"Measured current: {I_meas:.3e} A" if I_meas else "Measured current: N/A (need R_est)")
print(f"Recommended sensitivity: {recommended_sens:.3e} V")

# Show the calculation
print(f"\nCalculation details:")
print(f"V_sample = (X + jY) / Gv_eff = ({signal_chain.X()} + j{signal_chain.Y()}) / {signal_chain._gv_eff()} = {V_sample}")
if I_meas:
    print(f"I_meas = |V_sample| / R_est = {abs(V_sample):.3e} / {signal_chain.R_est()} = {I_meas:.3e} A")

## Signal Inversion

Demonstrate how signal inversion affects the calculations:

In [ ]:
print("=== Signal Inversion Demo ===")

# Show current effective values
print(f"Without inversion:")
print(f"Gm_eff = {signal_chain._gm_eff():.3e} A/V")
print(f"Gv_eff = {signal_chain._gv_eff():.1f} V/V")

# Enable V→I inversion
signal_chain.vi_invert.set(True)
print(f"\nWith V→I inversion enabled:")
print(f"Gm_eff = {signal_chain._gm_eff():.3e} A/V")
print(f"Gv_eff = {signal_chain._gv_eff():.1f} V/V")

# Set current with inversion - should result in negative source voltage
I_target = 1e-6
V_before_inversion = signal_chain.excitation_v_ac()
signal_chain.set_I_target(I_target)
V_after_inversion = signal_chain.excitation_v_ac()

print(f"\nSetting I_target = {I_target:.3e} A with inversion:")
print(f"Source voltage changed from {V_before_inversion:.3e} V to {V_after_inversion:.3e} V")
print(f"Commanded current: {signal_chain.get_I_cmd():.3e} A")

# Reset inversion for consistency
signal_chain.vi_invert.set(False)
signal_chain.preamp_invert.set(False)

## System Summary

Get a complete overview of the signal chain state:

In [ ]:
print("=== Final System Summary ===")

# Reset to a clean state
signal_chain.vi_invert.set(False)
signal_chain.preamp_invert.set(False)
signal_chain.set_I_target(1e-6)  # 1 µA

# Print complete summary
print(signal_chain.get_topology_summary())

print("\n=== Key Features Demonstrated ===")
print("✓ Single current setpoint parameter (I_set) with automatic voltage calculation")
print("✓ Open-loop current control via transconductance (gm)")
print("✓ Coupled frequency control for source and lock-in")
print("✓ Guard functions for input overload protection")
print("✓ Derived physics parameters (I_cmd, I_meas, V_sample_ac_meas)")
print("✓ Signal inversion support")
print("✓ Manual parameter snapshots for operator settings")
print("✓ Modular architecture with abstract bases and concrete nodes")